In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
from torchvision import models, transforms
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


In [ ]:
train_df = pd.read_csv("../data/processed/train_clean.csv")
test_df  = pd.read_csv("../data/processed/test_clean.csv")

TRAIN_IMG_DIR = "../data/images/train"
TEST_IMG_DIR  = "../data/images/test"


In [ ]:
img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    )
])


In [ ]:
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()   # removes final classification layer
resnet = resnet.to(device)
resnet.eval()


In [ ]:
def extract_image_features(df, img_dir):
    features = []
    ids = []

    missing = 0

    with torch.no_grad():
        for _, row in tqdm(df.iterrows(), total=len(df)):
            img_path = os.path.join(img_dir, f"{row['id']}.png")

            if not os.path.exists(img_path):
                missing += 1
                continue

            image = Image.open(img_path).convert("RGB")
            image = img_transform(image).unsqueeze(0).to(device)

            embedding = resnet(image)
            embedding = embedding.cpu().numpy().flatten()

            features.append(embedding)
            ids.append(row["id"])

    print(f"Skipped {missing} missing images")
    return np.array(features), ids

train_img_features, train_ids = extract_image_features(
    train_df, TRAIN_IMG_DIR
)

print(train_img_features.shape)



In [ ]:
test_img_features, test_ids = extract_image_features(
    test_df, TEST_IMG_DIR
)

print(test_img_features.shape)


In [ ]:
os.makedirs("../data/embeddings", exist_ok=True)

np.save("../data/embeddings/train_img_features.npy", train_img_features)
np.save("../data/embeddings/test_img_features.npy", test_img_features)


In [ ]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


In [ ]:
# Load cleaned tabular data
train_df = pd.read_csv("../data/processed/train_clean.csv")
test_df  = pd.read_csv("../data/processed/test_clean.csv")

# Load image embeddings
train_img = np.load("../data/embeddings/train_img_features.npy")
test_img  = np.load("../data/embeddings/test_img_features.npy")


In [ ]:
# Convert image embeddings to DataFrame
train_img_df = pd.DataFrame(train_img)
train_img_df["id"] = train_df.iloc[:len(train_img)]["id"].values

test_img_df = pd.DataFrame(test_img)
test_img_df["id"] = test_df.iloc[:len(test_img)]["id"].values


In [ ]:
train_df = train_df.merge(train_img_df, on="id", how="inner")
test_df  = test_df.merge(test_img_df, on="id", how="inner")


In [ ]:
tabular_features = [
    "bedrooms", "bathrooms", "sqft_living", "sqft_lot",
    "floors", "waterfront", "view", "condition", "grade",
    "sqft_above", "sqft_basement", "lat", "long",
    "sqft_living15", "sqft_lot15"
]

X_tab = train_df[tabular_features]
y = train_df["price"]


In [ ]:
X_tab_train, X_tab_val, y_train, y_val = train_test_split(
    X_tab, y, test_size=0.2, random_state=42
)


In [ ]:
#Model[A] tabular only

In [ ]:
scaler = StandardScaler()
X_tab_train_scaled = scaler.fit_transform(X_tab_train)
X_tab_val_scaled   = scaler.transform(X_tab_val)

lr = LinearRegression()
lr.fit(X_tab_train_scaled, y_train)

y_pred_tab = lr.predict(X_tab_val_scaled)
mse_tab = mean_squared_error(y_val, y_pred_tab)
rmse_tab = np.sqrt(mse_tab)
# rmse_tab = mean_squared_error(y_val, y_pred_tab, squared=False)
r2_tab = r2_score(y_val, y_pred_tab)

print("TABULAR ONLY RMSE:", rmse_tab)
print("TABULAR ONLY R²:", r2_tab)


In [ ]:
residuals = y_val - y_pred_tab

plt.hist(residuals, bins=50)
plt.title("Residual Distribution")
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_val, y_pred_tab)
print("mean absolute error(tab) : ",mae)

In [ ]:
#Model [B]

In [ ]:
image_features = [col for col in train_df.columns if col not in tabular_features + ["id", "price"]]

X_img = train_df[image_features].select_dtypes(include=["number"])
X_tab = train_df[tabular_features].select_dtypes(include=["number"])
X_multi = np.hstack([
    scaler.fit_transform(X_tab),
    X_img.values
])


In [ ]:
print("X_tab shape:", X_tab.shape)
print("X_img shape:", X_img.shape)
print("X_img dtypes:\n", X_img.dtypes.value_counts())


In [ ]:
X_train_m, X_val_m, y_train, y_val = train_test_split(
    X_multi, y, test_size=0.2, random_state=42
)


In [ ]:
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=20,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train_m, y_train)
y_pred_multi = rf.predict(X_val_m)

# Metrics
rmse_multi = np.sqrt(mean_squared_error(y_val, y_pred_multi))
r2_multi = r2_score(y_val, y_pred_multi)

print("MULTIMODAL RMSE:", rmse_multi)
print("MULTIMODAL R²:", r2_multi)


In [ ]:
residuals = y_val - y_pred_multi

plt.hist(residuals, bins=50)
plt.title("Residual Distribution")
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_val, y_pred_multi)
print("mean absolute error(multi) : ",mae)

In [ ]:
!pip uninstall numpy -y
!pip install numpy==1.26.4


In [ ]:
## Phase 6: Explainability with Grad-CAM
!pip install opencv-python



In [ ]:
import torch
import torch.nn.functional as F
import cv2
import numpy as np
# import matplotlib.pyplot as plt
from torchvision import models
from PIL import Image


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet = models.resnet18(pretrained=True)
resnet = resnet.to(device)
resnet.eval()


In [ ]:
target_layer = resnet.layer4[-1]


In [ ]:
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output

        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0]

        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)

    def generate(self, input_tensor):
        output = self.model(input_tensor)
        score = output.mean()  # regression proxy
        self.model.zero_grad()
        score.backward()

        weights = self.gradients.mean(dim=[2, 3], keepdim=True)
        cam = (weights * self.activations).sum(dim=1)

        cam = F.relu(cam)
        cam = cam - cam.min()
        cam = cam / cam.max()

        return cam.squeeze().detach().cpu().numpy()


In [ ]:
from torchvision import transforms

img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    )
])

def load_image(path):
    img = Image.open(path).convert("RGB")
    tensor = img_transform(img).unsqueeze(0).to(device)
    return img, tensor


In [ ]:
gradcam = GradCAM(resnet, target_layer)

# Pick ANY image (prefer expensive & cheap examples)
sample_id = train_df.sample(1)["id"].values[0]
img_path = f"../data/images/train/{sample_id}.png"

original_img, input_tensor = load_image(img_path)
cam = gradcam.generate(input_tensor)


In [ ]:
def overlay_cam(img, cam):
    img = np.array(img)
    cam = cv2.resize(cam, (img.shape[1], img.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)
    return overlay

overlay = overlay_cam(original_img, cam)

plt.figure(figsize=(6,6))
plt.imshow(overlay)
plt.axis("off")
plt.title(f"Grad-CAM Visualization (ID {sample_id})")
plt.show()
